<pre>
Copyright 2021-2023 Boris Shminke

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
</pre>

In [1]:
# we will prove a very basic statement from group theory
from gym_saturation.utils import MOCK_TPTP_PROBLEM

In [2]:
# "any idempotent item is the identity"
!cat {MOCK_TPTP_PROBLEM}

cnf(associativity, axiom, mult(X, mult(Y, Z)) = mult(mult(X, Y), Z)).
cnf(left_identity, axiom, mult(e, X) = X).
cnf(left_inverse, axiom, mult(inv(X), X) = e).
cnf(idempotent_element, hypothesis, mult(a, a) = a).
cnf(negated_conjecture, negated_conjecture, ~ a = e).


In [3]:
# we have a vampire installed in this container
!vampire --version

Vampire 4.7 (commit 2d02e4655 on 2022-07-11 21:15:24 +0200)
Linked with Z3 4.8.13.0 f03d756e086f81f2596157241e0decfb1c982299 z3-4.8.4-5390-gf03d756e0


In [4]:
# then we create and reset a Gymnasium environment
import gymnasium as gym

prover_name = "Vampire"
env = gym.make(f"{prover_name}-v0")
observation, info = env.reset()
# we can render the environment in a human readable form
print("starting hypotheses:")
env.render()

starting hypotheses:
cnf(1, lemma, mult(X0,mult(X1,X2)) = mult(mult(X0,X1),X2), inference(input, [], [])).
cnf(2, lemma, mult(e,X0) = X0, inference(input, [], [])).
cnf(3, lemma, e = mult(inv(X0),X0), inference(input, [], [])).
cnf(4, lemma, a = mult(a,a), inference(input, [], [])).
cnf(5, lemma, e != a, inference(input, [], [])).


In [5]:
def age_agent_episode(env: gym.Env) -> tuple[float, int, gym.core.ObsType]:
    """
    Interact with a given environment using the 'age' agent.

    'Age' agent always selects clauses for inference
    in the order they appeared in current proof attempt

    :param env: an environment
    :returns: a tuple of
        (number of steps in the episode, its gain, and the last observation)
    """
    action, num_steps, gain = 0, 0, 0
    terminated, truncated = False, False
    observation, info = env.reset()
    while not (terminated or truncated):
        # sometimes a prover can process an action automatically
        if observation["action_mask"][action] == 1.0:
            observation, reward, terminated, truncated, info = env.step(action)
            gain += reward
            num_steps += 1
        action += 1
    return gain, num_steps, observation

In [6]:
gain, num_steps, last_observation = age_agent_episode(env)
if gain > 0:
    print(f"Proof found in {num_steps} steps")

Proof found in 6 steps


In [7]:
# the package also provides a utility function
# for extracting only clauses which became parts of the proof
# (some steps might be unnecessary to find the proof)
from gym_saturation.utils import get_tstp_proof

print(get_tstp_proof(last_observation["real_obs"]))

cnf(1, lemma, mult(X0,mult(X1,X2)) = mult(mult(X0,X1),X2), inference(input, [], [])).
cnf(10, lemma, mult(inv(X0),mult(X0,X1)) = mult(e,X1), inference(superposition, [], [1, 3])).
cnf(2, lemma, mult(e,X0) = X0, inference(input, [], [])).
cnf(11, lemma, mult(inv(X0),mult(X0,X1)) = X1, inference(forward_demodulation, [], [10, 2])).
cnf(4, lemma, a = mult(a,a), inference(input, [], [])).
cnf(17, lemma, a = mult(inv(a),a), inference(superposition, [], [11, 4])).
cnf(3, lemma, e = mult(inv(X0),X0), inference(input, [], [])).
cnf(19, lemma, e = a, inference(forward_demodulation, [], [17, 3])).
cnf(5, lemma, e != a, inference(input, [], [])).
cnf(20, lemma, $false, inference(subsumption_resolution, [], [19, 5])).


In [8]:
# we have an iProver in this container
# (built from https://gitlab.com/korovin/iprover/-/commit/32c76fc0ec665a06324d11dc9d4bf902871d5f15)
!iproveropt --help | head


%---------------- iProver v3.7 (post CASC-J11 2022) ----------------%

iproveropt [options] [filename]



%---------------- Input Options ----------------%

  --out_options <none|control|all>


In [9]:
prover_name = "iProver"

In [10]:
import nest_asyncio

# iProver runs asynchronously with the main Python thread
# so we need to enable nested async loops (one is where Jupyter is running)
nest_asyncio.apply()

In [11]:
# and we can do the same experiment as we did with Vampire
env = gym.make(f"{prover_name}-v0")
gain, num_steps, last_observation = age_agent_episode(env)
# the number of steps is different, because this version of iProver
# relies on superposition instead of resolution
if gain > 0:
    print(f"Proof found in {num_steps} steps")

Proof found in 11 steps


In [12]:
print(get_tstp_proof(last_observation["real_obs"]))

cnf(c_49, lemma, mult(mult(X0,X1),X2)=mult(X0,mult(X1,X2)), inference(input, [], [])).
cnf(c_62, lemma, mult(inv(X0),mult(X0,X1))=mult(e,X1), inference(superposition, [], [c_51, c_49])).
cnf(c_50, lemma, mult(e,X0)=X0, inference(input, [], [])).
cnf(c_70, lemma, mult(inv(X0),mult(X0,X1))=X1, inference(demodulation, [], [c_62, c_50])).
cnf(c_52, lemma, mult(a,a)=a, inference(input, [], [])).
cnf(c_74, lemma, mult(inv(a),a)=a, inference(superposition, [], [c_52, c_70])).
cnf(c_51, lemma, mult(inv(X0),X0)=e, inference(input, [], [])).
cnf(c_85, lemma, e=a, inference(demodulation, [], [c_74, c_51])).
cnf(c_53, lemma, e!=a, inference(input, [], [])).
cnf(c_86, lemma, $false, inference(forward_subsumption_resolution, [], [c_85, c_53])).
